# 🧊 Iceberg workshop
Before we dive into the data, make sure you're using the same virtual environment as you did during setup. If you're using `uv`, that’s probably `.venv`!

## 🔍 Peek into the Iceberg catalog

Alright, let's take a little detour behind the scenes. We’re going to open the catalog database and see what Iceberg knows about our tables.

Here’s how you can list all the tables in the catalog:

In [ ]:
import sqlite3


with sqlite3.connect('../dlt_portable_data_lake_demo/_data/dev/local/catalog.db') as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
tables

Cool, we’re in the right place! Now let’s dig a little deeper.
You'll get a neat table showing metadata locations for each Iceberg table. It’s like peeking into the map that shows us where the treasure is buried 🗺️✨

In [ ]:
import pandas as pd


with sqlite3.connect('../dlt_portable_data_lake_demo/_data/dev/local/catalog.db') as conn:
    df = pd.read_sql_query("SELECT * FROM iceberg_tables", conn)
df

### 🎯 Mini Exercise: What’s the schema of the `stores` table?

Every Iceberg table comes with metadata that describes its schema. Your mission: figure out what the `stores` table looks like under the hood. Here's a little skeleton to help you get started:

In [ ]:
import json
import urllib.parse

file_uri = # put your code here
path_to_metadata = urllib.parse.urlparse(file_uri).path

with open(path_to_metadata, 'r') as file:
    data = json.load(file)
    
# print out the schema of the table

🧠 Stuck? Don’t worry, you can peek at the answer below:

<details>
<summary>Answer</summary>
    
```python
import json
import urllib.parse

file_uri = df.loc[2, "metadata_location"]
path_to_metadata = urllib.parse.urlparse(file_uri).path
with open(path_to_metadata, 'r') as file:
    data = json.load(file)
    
data["schemas"]
```
</details>

## 🔓 Accessing the Data – The Fun Begins 🎉

We’ve explored the catalog, now let’s actually use the data! Thanks to uv, your `dlt.yml` project is packaged like a proper Python module, so importing it is very easy.

In [ ]:
import dlt_portable_data_lake_demo

Now let’s list out the available datasets and tables:

In [ ]:
dataset_catalog = dlt_portable_data_lake_demo.catalog(profile="dev")
print("DATASETS")
print(dataset_catalog)

print("TABLES")
print(dataset_catalog.jaffle_shop_dataset)

To grab a specific table as a DataFrame, just do this:

```python
df =  dataset_catalog.jaffle_shop_dataset.<name_of_the_table>.df()
```

### 🎯 Mini Exercise: What kinds of products are sold in the Jaffle Shop?

Get the types of products straight from the products table!


In [ ]:
products = 
print()

<details>
    
<summary>Answer</summary>

```python
products = dataset_catalog.jaffle_shop_dataset.products.df()
print(products["type"].unique())
```

</details>

In [ ]:
products = dataset_catalog.jaffle_shop_dataset.orders.df()
products

## Access data with Ibis 🧙‍♂️ 

Say hello to [ibis](https://ibis-project.org/) – the lazy, expressive data wizard! 🪄 You can use Ibis to write pandas-like queries, and they’ll only run when you ask for the data. This library is open-source and can be used with most of the databases. Here’s how you use it with our catalog:


```python
ibis_statement = dataset_catalog.jaffle_shop_dataset.<name_of_the_table>.<any_ibis_method>()
result = ibis_statement.df()
```

In this case, Ibis will be used to generate SQL statement executed via duckdb on a view of Iceberg table 🤯

### 🎯 Mini Exercise: How many perishable vs non-perishable supplies do we have?

Hint: You’ll want to group by the perishable column and count the number of items in each group. Use [ibis documentation](https://ibis-project.org/tutorials/coming-from/pandas#group-by)!

In [ ]:
supplies = ...

<details>
    
<summary>Answer</summary>

```python
supplies = dataset_catalog.jaffle_shop_dataset.supplies
supplies.aggregate(by="perishable", count=supplies.id.count()).df()
```

</details>

Let’s look at just the perishable ones:

In [ ]:
supplies = dataset_catalog.jaffle_shop_dataset.supplies
perishable = supplies.filter(supplies.perishable)
perishable.df()

## Access Iceberg tables with SQL

If you’re more of a SQL fan, don’t worry – we got you! With the `sql_client` provided by dlt, you can run SQL directly on your Iceberg views:


In [ ]:
import duckdb

ds = dataset_catalog.jaffle_shop_dataset
with ds.sql_client as c:
    c.create_views_for_all_tables()
    conn: duckdb.DuckDBPyConnection = c.native_connection
    print(conn.sql("SHOW TABLES;"))
    print(conn.sql("SELECT * FROM stores"))

SQL + Iceberg + DuckDB = a triple threat combo 💪

## 🧊 Going full Iceberg with PyIceberg

Finally, we can get pyiceberg access to our data lake and utilize all our knowledge from the first part of the workshop!

In [ ]:
# get direct pyiceberg access to the datalake
iceberg_table = dataset_catalog.jaffle_shop_dataset.table_client.load_open_table("iceberg", "customers")
print(iceberg_table.location())
print(iceberg_table.scan().to_arrow())

# show catalog info
iceberg_catalog = dataset_catalog.jaffle_shop_dataset.table_client.get_open_table_catalog("iceberg")
print(iceberg_catalog.list_namespaces())

### 🏆 Final Exercise: Who’s the best customer?

Using any of the data access methods (Python, Ibis, SQL, or PyIceberg), find the name of the customer who placed the most orders.

## Bonus: Running transformations on DuckDB views

The next section is only relevant if you performed the Step 6 "Run SQL Workflows with dbt"!

Let's explore the resulting reports dataset:

In [ ]:
import dlt_portable_data_lake_demo
dataset_catalog = dlt_portable_data_lake_demo.catalog(profile="dev")
print(dataset_catalog.reports_dataset)

In [ ]:
dataset_catalog.reports_dataset.dim_orders.df()

In [ ]:
dataset_catalog.reports_dataset.dim_customers.df()